<a href="https://colab.research.google.com/github/vaTomas/lyric-video/blob/notebooks/notebooks/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Startup

### Clone Repository

In [ ]:
!git clone https://github.com/vaTomas/lyric-video.git

### Install Dependencies

In [ ]:
%pip install '/content/lyric-video/'

### Make Directories
This block will create folders on where inputs can be placed.

In [ ]:
import os

media_base_dir = '/content/user_media'

if not os.path.exists(media_base_dir):
    os.makedirs(media_base_dir)
    print(f"Created directory: {media_base_dir}")

subdirectories = ['subtitles', 'music', 'images', 'font', 'outputs']

for subdir in subdirectories:
    subdir_path = os.path.join(media_base_dir, subdir)
    if not os.path.exists(subdir_path):
        os.makedirs(subdir_path)
        print(f"Created directory: {subdir_path}")
    else:
        print(f"Directory already exists: {subdir_path}")

## Configuration

You can get your font files from https://www.dafont.com/.
You can get your subtitles from https://www.rentanadviser.com/subtitles/subtitle-finder-for-youtube-music-videos.aspx

In [ ]:
# Resolution
display_width = 1920
display_height = 1080

# Text Font
font_size = 180
font_style = '/content/user_media/font/font.otf'
font_color = 'black'

# Background
background_color = 'white'

# Subtitles
subtitles_path = '/content/user_media/subtitles/subtitle.srt'

# Images
image_size = 400

# Output
output_directory = '/content/user_media/outputs/'

### Preview

In [ ]:
from PIL import ImageFont, Image
from elements import TextElement

sample_text = 'Hello World!'

font_obj = ImageFont.truetype(font_style, font_size)
text_element = TextElement(
    text=sample_text,
    font=font_obj,
    text_color=font_color,
    position = (display_width / 2, display_height / 2)
)

preview_image = Image.new('RGB', (display_width, display_height), background_color)
text_element.draw(preview_image)
display(preview_image)

##Subtitles
Upload subtitle file in the 'user_media/subtitles' folder.

The code block below extracts subtitle information from the .srt file and converts them into Subtitle objects.

In [ ]:
from subtitle import get_subtitles

subtitles = get_subtitles(subtitles_path)

print(f'Extracted {len(subtitles)} subtitles.')

##Objects

###Create Scene Objects

In [ ]:
from Scene import Scene

canvas_size = int((2 * (display_width * display_height) * len(subtitles)) ** (1/2))
resolution = (display_width, display_height)

scene_text = Scene(canvas_size, canvas_size)

print(f'Created scene with canvas size {canvas_size}.')

###Create Text Objects

Place your font file to '/content/usermedia/font'

You can get your font files from https://www.dafont.com/.

In [ ]:
from elements import TextElement

scene_text.elements = []
for subtile in subtitles:
        text_element = TextElement(
            text=subtile.content, start=subtile.start, end=subtile.end, font=font_obj
        )
        scene_text.add_object(text_element)

print(f'Created {len(scene_text.elements)} text objects inside scene.')

In [ ]:
min_element_width = canvas_size
for element in scene_text.elements:
    element_width = element.object_box_size[0]
    min_element_width = min(element_width, min_element_width)
view_padding = (max(resolution) - min_element_width) / 2

print(f'Resolution: {resolution}.')
print(f'Minimum element width: {min_element_width}.')
print(f'Calculated view padding: {view_padding}.')

In [ ]:
import math

for element in scene_text.elements:
    index = scene_text.element_index(element)
    if index == 0:
        scene_text.move_random(
            element, constrain_to_artboard=True, artboard_margin=view_padding
        )
        continue

    if not scene_text.move_next(
        element,
        scene_text.elements[index - 1],
        minimum_distance=view_padding + image_size * math.sqrt(2) / 2,
        artboard_margin=view_padding,
    ):
        break

print(f"Placed {len(scene_text.elements)} of {len(subtitles)} text lyrics.")
if len(scene_text.elements) != len(subtitles):
    f"Only placed {len(scene_text.elements)} of {len(subtitles)} text lyrics"

## Render Images

In [ ]:
import datetime

datetime_now = datetime.datetime.now()
datetime_now = datetime_now.strftime("%Y_%m_%d_%H%M%S")

scene_text.image = Image.new("RGBA", (canvas_size, canvas_size), None)

scene_text.draw_objects()
scene_text_path = os.path.join(
        output_directory, f"{datetime_now}_image_text.png"
    )
scene_text.save(scene_text_path)

print(f"Saved text scene to {scene_text_path}.")
display(scene_text.image)